In [67]:
import sklearn.linear_model
import numpy as np
import pandas as pd
import os
from SoundLights.features_groups import  ARAUS_features, Freesound_features, mix_features, masker_features

from sklearn.preprocessing import RobustScaler
import numpy as np

# Create a RobustScaler object
scaler = RobustScaler()

normalise = lambda X: (X-np.mean(X,axis=0,keepdims=True))/np.std(X,axis=0,keepdims=True) # Normalise an (n,p) numpy array to mean 0, variance 1.
clip = lambda x, x_min = -1, x_max = 1: np.where(np.where(x < x_min,x_min,x) > x_max, x_max, np.where(x < x_min,x_min,x)) # Clip an array to values between x_min and x_max.
def normalize_columns(data):
    # Calculate mean and standard deviation for each column
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    
    # Normalize each column separately
    normalized_data = (data - mean) / (std)
    
    return normalized_data, mean, std

def normalize_columns_minmax(data):
    # Calculate mean and standard deviation for each column
    min = np.min(data, axis=0)
    max = np.max(data, axis=0)
    
    # Normalize each column separately
    normalized_data = (data - min) / (max-min)
    
    return normalized_data, min, max

def normalize_columns_log(data):
    log_transformed_data = np.log(data + 1)

    return log_transformed_data

In [68]:
responses_Freesound= pd.read_csv(os.path.join('..','data/csv_files','SoundLights_Freesound.csv')) #, dtype = {'participant':str}
responses_Freesound=responses_Freesound.drop("info.file", axis=1)
responses_Freesound=responses_Freesound.drop("info.participant", axis=1)


In [69]:
responses_Mix= pd.read_csv(os.path.join('..','data/csv_files','SoundLights_mix.csv')) #, dtype = {'participant':str}
responses_Mix=responses_Mix.drop("info.file", axis=1)
responses_Mix=responses_Mix.drop("info.participant", axis=1)

In [70]:
responses_complete= pd.read_csv(os.path.join('..','data/csv_files','SoundLights_complete.csv')) #, dtype = {'participant':str}
responses_complete=responses_complete.drop("info.file", axis=1)
responses_complete=responses_complete.drop("info.participant", axis=1)

In [71]:
responses_ARAUS= pd.read_csv(os.path.join('..','data/csv_files','SoundLights_ARAUS.csv'), dtype = {'info.participant':str}) #, dtype = {'participant':str}
responses_ARAUS=responses_ARAUS.drop("info.file", axis=1)
responses_ARAUS=responses_ARAUS.drop("info.participant", axis=1)

# Drop columns that contain all zero values
# Store column names before dropping
columns_before = responses_ARAUS.columns.tolist()
# Drop zero-columns
responses_ARAUS = responses_ARAUS.loc[:, (responses_ARAUS != 0).any(axis=0)]
# Store column names after dropping
columns_after = responses_ARAUS.columns.tolist()
# Determine which columns were dropped
columns_dropped = [col for col in columns_before if col not in columns_after]
# Drop those columns from ARAUS_features
ARAUS_features = [col for col in ARAUS_features if col not in columns_dropped]
print(ARAUS_features)
#MAskers colum, increase values
responses_ARAUS["info.masker_bird"]=responses_ARAUS["info.masker_bird"]*5
responses_ARAUS["info.masker_construction"]=responses_ARAUS["info.masker_construction"]*5
responses_ARAUS["info.masker_traffic"]=responses_ARAUS["info.masker_traffic"]*5
responses_ARAUS["info.masker_water"]=responses_ARAUS["info.masker_water"]*5
responses_ARAUS["info.masker_wind"]=responses_ARAUS["info.masker_wind"]*5

['ARAUS.sharpness.avg', 'ARAUS.sharpness.max', 'ARAUS.sharpness.p05', 'ARAUS.sharpness.p10', 'ARAUS.sharpness.p20', 'ARAUS.sharpness.p30', 'ARAUS.sharpness.p40', 'ARAUS.sharpness.p50', 'ARAUS.sharpness.p60', 'ARAUS.sharpness.p70', 'ARAUS.sharpness.p80', 'ARAUS.sharpness.p90', 'ARAUS.sharpness.p95', 'ARAUS.loudness.avg', 'ARAUS.loudness.max', 'ARAUS.loudness.p05', 'ARAUS.loudness.p10', 'ARAUS.loudness.p20', 'ARAUS.loudness.p30', 'ARAUS.loudness.p40', 'ARAUS.loudness.p50', 'ARAUS.loudness.p60', 'ARAUS.loudness.p70', 'ARAUS.loudness.p80', 'ARAUS.loudness.p90', 'ARAUS.loudness.p95', 'ARAUS.fluctuation.avg', 'ARAUS.fluctuation.max', 'ARAUS.fluctuation.p05', 'ARAUS.fluctuation.p10', 'ARAUS.fluctuation.p20', 'ARAUS.fluctuation.p30', 'ARAUS.fluctuation.p40', 'ARAUS.fluctuation.p50', 'ARAUS.fluctuation.p60', 'ARAUS.fluctuation.p70', 'ARAUS.fluctuation.p80', 'ARAUS.fluctuation.p90', 'ARAUS.fluctuation.p95', 'ARAUS.LA.avg', 'ARAUS.LA.min', 'ARAUS.LA.max', 'ARAUS.LA.p05', 'ARAUS.LA.p10', 'ARAUS.LA

### Adjust alpha

In [72]:
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.linear_model import ElasticNet
# Define your ElasticNet model with specific hyperparameters
alpha = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1] 
l1_ratio = 0.5
print('     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------')

for value in alpha:

    model = ElasticNet(alpha=value, l1_ratio=l1_ratio, selection="random")
    #print(f'Investigating performance of {model} model...')

    MSEs_train = []
    MSEs_val = []
    MSEs_test = []
    MEs_train = []
    MEs_val = []
    MEs_test = []

    
    for val_fold in [1,2,3,4,5]:

        # Extract dataframes
        df_train = responses_ARAUS[(responses_ARAUS['info.fold'] != val_fold) & (responses_ARAUS['info.fold'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
        df_val   = responses_ARAUS[responses_ARAUS['info.fold'] == val_fold]
        df_test  = responses_ARAUS[responses_ARAUS['info.fold'] == 0].groupby(['info.soundscape','info.masker','info.smr']).mean() # For the test set, the same 48 stimuli were shown to all participants so we take the mean of their ratings as the ground truth

        # Get ground-truth labels
        Y_train = df_train['info.E_ground_truth'].values
        Y_val = df_val['info.E_ground_truth'].values
        Y_test = df_test['info.E_ground_truth'].values


        # Get features normalized_data = (data - min) / (max-min)
        X_train = df_train[ARAUS_features+masker_features].values
        X_val =df_val[ARAUS_features+masker_features].values
        X_test = df_test[ARAUS_features+masker_features].values
        """ X_train, mean, std = normalize_columns(df_train[ARAUS_features+masker_features].values)
        X_val =((df_val[ARAUS_features+masker_features].values)- mean) / (std)
        X_test = ((df_test[ARAUS_features+masker_features].values)- mean) / (std)  """
        
        
        """ scaler.fit(df_train[ARAUS_features].values)

        # Transform the data using the scaler
        X_train = scaler.transform(df_train[ARAUS_features].values)
        X_val =scaler.transform(df_val[ARAUS_features].values)
        X_test = scaler.transform(df_test[ARAUS_features].values) """

        # Fit model
        X_LR = model.fit(X_train, Y_train)

        # Get MSEs
        MSE_train = np.mean((clip(X_LR.predict(X_train)) - Y_train)**2)
        MSE_val = np.mean((clip(X_LR.predict(X_val)) - Y_val)**2)
        MSE_test = np.mean((clip(X_LR.predict(X_test)) - Y_test)**2)
        ME_train = np.mean(np.abs(clip(X_LR.predict(X_train)) - Y_train))
        ME_val = np.mean(np.abs(clip(X_LR.predict(X_val)) - Y_val))
        ME_test = np.mean(np.abs(clip(X_LR.predict(X_test)) - Y_test))

        # Add metrics
        MSEs_train.append(MSE_train)
        MSEs_val.append(MSE_val)
        MSEs_test.append(MSE_test)
        MEs_train.append(ME_train)
        MEs_val.append(ME_val)
        MEs_test.append(ME_test)

        #print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {ME_train:.4f} | {ME_val:.4f} | {ME_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d} |')
    print("Parameters ",value, l1_ratio )
    print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f} | {np.mean(MEs_train):.4f} | {np.mean(MEs_val):.4f} | {np.mean(MEs_test):.4f} |')

     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------
Parameters  0.1 0.5
Mean | 0.1292 | 0.1323 | 0.0353 | 0.2960 | 0.2994 | 0.1493 |
Parameters  0.2 0.5
Mean | 0.1308 | 0.1329 | 0.0323 | 0.2983 | 0.3006 | 0.1433 |
Parameters  0.3 0.5
Mean | 0.1315 | 0.1333 | 0.0317 | 0.2996 | 0.3015 | 0.1441 |
Parameters  0.4 0.5
Mean | 0.1321 | 0.1337 | 0.0322 | 0.3007 | 0.3022 | 0.1472 |
Parameters  0.5 0.5
Mean | 0.1328 | 0.1342 | 0.0329 | 0.3018 | 0.3032 | 0.1510 |
Parameters  0.6 0.5
Mean | 0.1336 | 0.1348 | 0.0341 | 0.3030 | 0.3043 | 0.1552 |
Parameters  0.7 0.5
Mean | 0.1345 | 0.1356 | 0.0356 | 0.3044 | 0.3055 | 0.1596 |
P

### Adjust l1_ratio

In [74]:
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.linear_model import ElasticNet
# Define your ElasticNet model with specific hyperparameters
alpha = 0.2
l1_ratio = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1] 
print('     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------')

for value in l1_ratio:

    model = ElasticNet(alpha=alpha, l1_ratio=value, selection="random")
    #print(f'Investigating performance of {model} model...')

    MSEs_train = []
    MSEs_val = []
    MSEs_test = []
    MEs_train = []
    MEs_val = []
    MEs_test = []

    
    for val_fold in [1,2,3,4,5]:

        # Extract dataframes
        df_train = responses_ARAUS[(responses_ARAUS['info.fold'] != val_fold) & (responses_ARAUS['info.fold'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
        df_val   = responses_ARAUS[responses_ARAUS['info.fold'] == val_fold]
        df_test  = responses_ARAUS[responses_ARAUS['info.fold'] == 0].groupby(['info.soundscape','info.masker','info.smr']).mean() # For the test set, the same 48 stimuli were shown to all participants so we take the mean of their ratings as the ground truth

        # Get ground-truth labels
        Y_train = df_train['info.E_ground_truth'].values
        Y_val = df_val['info.E_ground_truth'].values
        Y_test = df_test['info.E_ground_truth'].values

        # Get features normalized_data = (data - min) / (max-min)
        X_train = df_train[ARAUS_features+masker_features].values
        X_val =df_val[ARAUS_features+masker_features].values
        X_test = df_test[ARAUS_features+masker_features].values
        """ X_train, min, max = normalize_columns_minmax(df_train[ARAUS_features].values)
        X_val =((df_val[ARAUS_features].values)- min) / (max-min)
        X_test = ((df_test[ARAUS_features].values)- min) / (max-min) """
        """ scaler.fit(df_train[ARAUS_features].values)

        # Transform the data using the scaler
        X_train = scaler.transform(df_train[ARAUS_features].values)
        X_val =scaler.transform(df_val[ARAUS_features].values)
        X_test = scaler.transform(df_test[ARAUS_features].values) """

        # Fit model
        X_LR = model.fit(X_train, Y_train)

        # Get MSEs
        MSE_train = np.mean((clip(X_LR.predict(X_train)) - Y_train)**2)
        MSE_val = np.mean((clip(X_LR.predict(X_val)) - Y_val)**2)
        MSE_test = np.mean((clip(X_LR.predict(X_test)) - Y_test)**2)
        ME_train = np.mean(np.abs(clip(X_LR.predict(X_train)) - Y_train))
        ME_val = np.mean(np.abs(clip(X_LR.predict(X_val)) - Y_val))
        ME_test = np.mean(np.abs(clip(X_LR.predict(X_test)) - Y_test))

        # Add metrics
        MSEs_train.append(MSE_train)
        MSEs_val.append(MSE_val)
        MSEs_test.append(MSE_test)
        MEs_train.append(ME_train)
        MEs_val.append(ME_val)
        MEs_test.append(ME_test)

        #print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {ME_train:.4f} | {ME_val:.4f} | {ME_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d} |')
    print("Parameters ",alpha, value )
    print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f} | {np.mean(MEs_train):.4f} | {np.mean(MEs_val):.4f} | {np.mean(MEs_test):.4f} |')
    """ coefficients = X_LR.coef_
    feature_importances = list(zip(ARAUS_features+masker_features, coefficients))
    feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)
    list_features=[]
    for feature, importance in feature_importances:
        if float(importance)!=0:
            print(f"{feature}: {importance}")
            list_features.append(feature)
    print("LIST ",list_features) """

     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------
Parameters  0.2 0.1
Mean | 0.1264 | 0.1310 | 0.0403 | 0.2922 | 0.2975 | 0.1615 |
Parameters  0.2 0.2
Mean | 0.1285 | 0.1319 | 0.0361 | 0.2951 | 0.2989 | 0.1509 |
Parameters  0.2 0.3
Mean | 0.1296 | 0.1325 | 0.0346 | 0.2965 | 0.2998 | 0.1478 |
Parameters  0.2 0.4
Mean | 0.1302 | 0.1328 | 0.0332 | 0.2976 | 0.3002 | 0.1448 |
Parameters  0.2 0.5
Mean | 0.1308 | 0.1329 | 0.0323 | 0.2983 | 0.3006 | 0.1433 |
Parameters  0.2 0.6
Mean | 0.1311 | 0.1331 | 0.0319 | 0.2989 | 0.3010 | 0.1433 |
Parameters  0.2 0.7
Mean | 0.1314 | 0.1333 | 0.0318 | 0.2994 | 0.3013 | 0.1438 |
P

In [76]:
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.linear_model import ElasticNet
# Define your ElasticNet model with specific hyperparameters
alpha = 0.2
l1_ratio = 0.5
print('     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------')



model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, selection="random")
#print(f'Investigating performance of {model} model...')

MSEs_train = []
MSEs_val = []
MSEs_test = []
MEs_train = []
MEs_val = []
MEs_test = []


for val_fold in [1,2,3,4,5]:

    # Extract dataframes
    df_train = responses_ARAUS[(responses_ARAUS['info.fold'] != val_fold) & (responses_ARAUS['info.fold'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
    df_val   = responses_ARAUS[responses_ARAUS['info.fold'] == val_fold]
    df_test  = responses_ARAUS[responses_ARAUS['info.fold'] == 0].groupby(['info.soundscape','info.masker','info.smr']).mean() # For the test set, the same 48 stimuli were shown to all participants so we take the mean of their ratings as the ground truth

    # Get ground-truth labels
    Y_train = df_train['info.E_ground_truth'].values
    Y_val = df_val['info.E_ground_truth'].values
    Y_test = df_test['info.E_ground_truth'].values

    # Get features normalized_data = (data - min) / (max-min)
    X_train = df_train[ARAUS_features+masker_features].values
    X_val =df_val[ARAUS_features+masker_features].values
    X_test = df_test[ARAUS_features+masker_features].values
    """ X_train, min, max = normalize_columns_minmax(df_train[ARAUS_features].values)
    X_val =((df_val[ARAUS_features].values)- min) / (max-min)
    X_test = ((df_test[ARAUS_features].values)- min) / (max-min) """
    """ scaler.fit(df_train[ARAUS_features].values)

    # Transform the data using the scaler
    X_train = scaler.transform(df_train[ARAUS_features].values)
    X_val =scaler.transform(df_val[ARAUS_features].values)
    X_test = scaler.transform(df_test[ARAUS_features].values) """

    # Fit model
    X_LR = model.fit(X_train, Y_train)

    # Get MSEs
    MSE_train = np.mean((clip(X_LR.predict(X_train)) - Y_train)**2)
    MSE_val = np.mean((clip(X_LR.predict(X_val)) - Y_val)**2)
    MSE_test = np.mean((clip(X_LR.predict(X_test)) - Y_test)**2)
    ME_train = np.mean(np.abs(clip(X_LR.predict(X_train)) - Y_train))
    ME_val = np.mean(np.abs(clip(X_LR.predict(X_val)) - Y_val))
    ME_test = np.mean(np.abs(clip(X_LR.predict(X_test)) - Y_test))

    # Add metrics
    MSEs_train.append(MSE_train)
    MSEs_val.append(MSE_val)
    MSEs_test.append(MSE_test)
    MEs_train.append(ME_train)
    MEs_val.append(ME_val)
    MEs_test.append(ME_test)

    #print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {ME_train:.4f} | {ME_val:.4f} | {ME_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d} |')
print("Parameters ",alpha, l1_ratio )
print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f} | {np.mean(MEs_train):.4f} | {np.mean(MEs_val):.4f} | {np.mean(MEs_test):.4f} |')
coefficients = X_LR.coef_
feature_importances = list(zip(ARAUS_features+masker_features, coefficients))
feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)
list_features=[]
for feature, importance in feature_importances:
    if float(importance)!=0:
        print(f"{feature}: {importance}")
        list_features.append(feature)
print("LIST ",list_features)

     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------
Parameters  0.2 0.5
Mean | 0.1308 | 0.1329 | 0.0323 | 0.2983 | 0.3006 | 0.1433 |
ARAUS.LA.max: 0.008389068033380282
ARAUS.energy_frequency.00630_0: 0.0046823517018782885
ARAUS.energy_frequency.00500_0: 0.0035934993776887453
ARAUS.loudness.max: 0.0015935251646821515
ARAUS.LA.p10: 0.0015271507806383247
ARAUS.LC.max: 0.0006861040617556041
ARAUS.energy_frequency.20000_0: -0.0006769021971379923
LIST  ['ARAUS.LA.max', 'ARAUS.energy_frequency.00630_0', 'ARAUS.energy_frequency.00500_0', 'ARAUS.loudness.max', 'ARAUS.LA.p10', 'ARAUS.LC.max', 'ARAUS.energy_frequency.20000_